In [31]:
import time
import pandas as pd
from census import Census
import altair as alt
import folium
import numpy as np
import geopandas as gpd

#import census tract shapefiles
tracts = gpd.read_file("data_sources/cb_2023_06_tract_500k.zip")
tracts = tracts[tracts["COUNTYFP"] == "001"]

In [32]:
#import disadvantaged community data and join
dac_df = pd.read_excel("data_sources/SB535DACresultsdatadictionary_F_2022.xlsx", sheet_name='SB535 tract all data (2022)')
dac_cols = {
    "Census Tract":"GEOID",
    "DAC category":"DAC_type"
}
dac_df = dac_df[dac_cols.keys()]
dac_df = dac_df.rename(columns=dac_cols)
#make GEOID the right type
dac_df["GEOID"] = '0' + dac_df['GEOID'].astype(str)

dac_tracts = tracts.merge(dac_df, on="GEOID")

In [34]:
#import precipitation
prec_df = pd.read_csv("data_sources/prec_2040.csv")

In [59]:
#filter by the simulation type
prec_df = prec_df[prec_df["simulation"] == "LOCA2_ACCESS-CM2_r1i1p1f1_historical+ssp245"]
#group by lat long and find the mean for the year
prec_group_2040 = prec_df.groupby(by=["lat", "lon"])["Precipitation_total"].mean().to_frame(name="Precipitation_mean").reset_index()
#prec_df.info()
prec_min = prec_group_2040["Precipitation_mean"].min()
prec_max = prec_group_2040["Precipitation_mean"].max()
prec_group_2040["prec_scaled"] = (prec_group_2040["Precipitation_mean"] - prec_min) / (prec_max - prec_min)

In [62]:
prec_group_2040[prec_group_2040["Precipitation_mean"].notnull()]

,lat,lon,Precipitation_mean,prec_scaled
9,37.484375,-122.078125,0.468981,0.082796
10,37.484375,-122.046875,0.440699,0.059202
11,37.484375,-122.015625,0.446087,0.063698
12,37.484375,-121.984375,0.479136,0.091267
13,37.484375,-121.953125,0.552060,0.152101
...,...,...,...,...
350,37.859375,-122.296875,0.983833,0.512290
351,37.859375,-122.265625,1.116366,0.622851
352,37.859375,-122.234375,1.296111,0.772795
379,37.890625,-122.296875,1.141335,0.643679


In [65]:
m = folium.Map(location=(37.6017, -121.8), zoom_start=10, tiles="cartodb positron")

dac_outlines = folium.GeoJson(
    dac_tracts,
    style_function=lambda feature: {
        "weight": 1,
        "color":"Pink"
    }
).add_to(m)

def plotDot(point):
    folium.CircleMarker(
        location=[point.lat,point.lon],
        radius=point.prec_scaled*6,
        weight=1
    ).add_to(m)

prec_group_2040.apply(plotDot, axis=1)

m